## End-to-End Scenario: Scenario Survival Analysis
Author: TI HA DB ML China - SAP HANA PAL Team

Date: 2020/06/18

In clinical trials or community trials, the effect of an intervention is assessed by measuring the number of subjects who have survived or are saved after that intervention over a period of time. We wish to measure the survival probability of Dukes’C colorectal cancer patients after treatment and evaluate statistically whether the patients who accept treatment can survive longer than those who are only controlled conservatively.


## 1.  Setup the Connection to SAP HANA
First, create a connetion to SAP HANA. To create a such connection, a config file, config/e2edata.ini is used to control the connection parameters.A sample section in the config file is shown below which includes HANA url, port, user and password information.<br>

###################<br>
[hana]<br>
url=host-url<br>
user=username<br>
passwd=userpassword<br>
port=3xx15<br>
###################<br>

In [1]:
from hana_ml.dataframe import ConnectionContext
from data_load_utils import DataSets, Settings
url, port, user, pwd = Settings.load_config("../../../config/e2edata.ini")
connection_context = ConnectionContext(url, port, user, pwd)

Connection status:

In [2]:
print(connection_context.connection.isconnected())

True


##  2. Dataset
This scenarios describes a clinical trial of 49 patients for the treatment of Dukes’C colorectal cancer. The following data shows the survival time in 49 patients with Dukes’C colorectal cancer who are randomly assigned to either linoleic acid or control treatment.

![](patient.png)

The + sign indicates censored data. Until 6 months after treatment, there are no deaths. The effect of the censoring is to remove from the alive group those that are censored. At time 6 months two subjects have been censored so the number alive just before 6 months is 23. There are two deaths at 6 months. Thus,

We now reduce the number alive (“at risk”) by two. The censored event at 9 months reduces the “at risk” set to 20. At 10 months there are two deaths. So the proportion surviving is 18/20 = 0.9, and the cumulative proportion surviving is 0.913*0.90 = 0.8217.

## 3. Implementation Steps

### Option 1: Kaplan-Meier Estimate
**Technology Background**

Kaplan-Meier estimate is one of the simplest way to measure the fraction of subjects living for a certain amount of time after treatment. The time starting from a defined point to the occurrence of a given event, for example death, is called as survival time.

**Step 1**

The trial data can then be loaded into table as follows:


In [3]:
import pandas as pd
from hana_ml.dataframe import create_dataframe_from_pandas

data = {'TIME':  [1,5,6,6,9,10,10,10,12,12,13,15,16,20,24,24,27,32,34,36,44,3,6,8,12,12,15,16,18,20,22,24,28,30,30,33,42],
        'STATUS': [0,0,1,1,0,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,1,0,0,1,0,1],
        'OCCURRENCES': [1,1,1,1,1,1,1,1,4,1,1,1,1,1,1,1,1,1,1,2,1,1,4,2,2,1,1,1,2,1,1,1,3,1,1,1,1],
        'GROUP': ["linoleic acid","linoleic acid","linoleic acid","linoleic acid","linoleic acid","linoleic acid",
                  "linoleic acid","linoleic acid","linoleic acid","linoleic acid","linoleic acid","linoleic acid",
                  "linoleic acid","linoleic acid","linoleic acid","linoleic acid","linoleic acid","linoleic acid",
                  "linoleic acid","linoleic acid","linoleic acid","control","control","control","control","control",
                  "control","control","control","control","control","control","control","control","control","control",
                  "control"] }

trial = pd.DataFrame (data, columns = ['TIME','STATUS','OCCURRENCES', 'GROUP'])
trial_df = create_dataframe_from_pandas(connection_context, pandas_df=trial, 
                                        table_name='PAL_TRIAL_DATA_TBL', force=True, replace=True)
trial_df.head(5).collect()


,TIME,STATUS,OCCURRENCES,GROUP
0,1,0,1,linoleic acid
1,5,0,1,linoleic acid
2,6,1,1,linoleic acid
3,6,1,1,linoleic acid
4,9,0,1,linoleic acid


**Step 2**

Input customer data and use the Kaplan-Meier function to get the survival estimates and log-rank test statistics.

To compare survival estimates produced from two groups, we use log-rank test. 
It is a hypothesis test to compare the survival distribution of two groups (some of the observations may be censored) 
and is used to test the null hypothesis that there is no difference between the populations (treatment group and control group)
in the probability of an event (here a death) at any time point. The methods are nonparametric in 
that they do not make assumptions about the distributions of survival estimates. 
The analysis is based on the times of events (here deaths). For each such time 
we calculate the observed number of deaths in each group and the number expected 
if there were in reality no difference between the groups. It is widely used in clinical trials 
to establish the efficacy of a new treatment in comparison with a control treatment when the measurement 
is the time to event (such as the time from initial treatment to death).

Because the log-rank test is purely a test of significance, it cannot provide an estimate of the size of the difference between the groups.



In [4]:
from hana_ml.algorithms.pal.stats import kaplan_meier_survival_analysis
result = kaplan_meier_survival_analysis(trial_df)
print(result[0].collect())

            GROUP  TIME  RISK_NUMBER  EVENT_NUMBER  PROBABILITY        SE  \
0         control   6.0           23             4     0.826087  0.079034   
1         control   8.0           19             2     0.739130  0.091561   
2         control  12.0           17             2     0.652174  0.099311   
3         control  20.0           10             1     0.586957  0.108705   
4         control  24.0            8             1     0.513587  0.117292   
5         control  30.0            4             1     0.385190  0.141785   
6         control  42.0            1             1     0.000000       NaN   
7   linoleic acid   6.0           23             2     0.913043  0.058753   
8   linoleic acid  10.0           20             2     0.821739  0.080917   
9   linoleic acid  12.0           17             4     0.628389  0.104766   
10  linoleic acid  24.0            8             1     0.549840  0.117482   
11  linoleic acid  32.0            5             1     0.439872  0.136043   

In [5]:
print(result[1].collect())

           GROUP  TOTAL_RISK  OBSERVED   EXPECTED  LOGRANK_STAT
0        control          24        12  10.714679      0.328949
1  linoleic acid          25        10  11.285321      0.328949


In [6]:
print(result[2].collect())

  STAT_NAME  STAT_VALUE
0    chiSqr    0.328949
1        df    1.000000
2   p-value    0.566278


###  Option 2: Weibull Distribution
**Technology Background**

Weibull distribution is often used for reliability and survival analysis. It is defined by 3 parameters: shape, scale, and location. Scale works as key to magnify or shrink the curve. Shape is the crucial factor to define how the curve looks like, as described below:

 - Shape = 1: The failure rate is constant over time, indicating random failure.
 
 - Shape < 1: The failure rate decreases over time.
 
 - Shape > 1: The failure rate increases over time.

**Step 1**

Get Weibull distribution and statistics from the linoleic acid treatment data:

In [7]:
cursor = connection_context.connection.cursor()
try:
     cursor.execute("DROP TABLE PAL_LINO_ACID_TBL")
except:
     pass

cursor.execute('CREATE COLUMN TABLE PAL_LINO_ACID_TBL (\"LEFT\" DOUBLE, \"RIGHT\" DOUBLE);')
values = [(1,None),
          (5,None),
          (6,6),
          (6,6),
          (9,None),
          (10,10),
          (10,10),
          (10,None),
          (12,12),
          (12,12),
          (12,12),
          (12,12),
          (12,None),
          (13,None),
          (15,None),
          (16,None),
          (20,None),
          (24,24),
          (24,None),
          (27,None),
          (32,32),
          (34,None),
          (36,None),
          (36,None),
          (44,None)]
try:
    cursor.executemany("INSERT INTO " +
                       "{} VALUES ({})".format('PAL_LINO_ACID_TBL',
                       ', '.join(['?']*len(values[0]))), values)
    connection_context.connection.commit()
finally:
    cursor.close()
linoleic_acid_df = connection_context.table("PAL_LINO_ACID_TBL")

print(linoleic_acid_df.head(5).collect())

   LEFT  RIGHT
0   1.0    NaN
1   5.0    NaN
2   6.0    6.0
3   6.0    6.0
4   9.0    NaN


Call Weibull Distribution function and show the results:

In [8]:
from hana_ml.algorithms.pal.stats import distribution_fit
result = distribution_fit(linoleic_acid_df, distr_type = "weibull", optimal_method = "maximum_likelihood", censored=True)
print(result[0].collect())

               NAME    VALUE
0  DISTRIBUTIONNAME  WEIBULL
1             SCALE  36.3069
2             SHAPE  1.40528


In [9]:
print(result[1].collect())

       STAT_NAME  STAT_VALUE
0  LOGLIKELIHOOD  -47.035827


**Step 2**

Get Weibull distribution and statistics from the control treatment data:

In [10]:
cursor = connection_context.connection.cursor()
try:
    cursor.execute("DROP TABLE PAL_CONTROL_TBL")
except:
    pass
cursor.execute('CREATE COLUMN TABLE PAL_CONTROL_TBL (\"LEFT\" DOUBLE, \"RIGHT\" DOUBLE);')
values = [(3,None),
          (6,6),
          (6,6),
          (6,6),
          (6,6),
          (8,8),
          (8,8),
          (12,12),
          (12,12),
          (12,None),
          (15,None),
          (16,None),
          (18,None),
          (18,None),
          (20,20),
          (22,None),
          (24,24),
          (28,None),
          (28,None),
          (28,None),
          (30,30),
          (30,None),
          (33,None),
          (42,42)]
try:
    cursor.executemany("INSERT INTO " +
                       "{} VALUES ({})".format('PAL_CONTROL_TBL',
                       ', '.join(['?']*len(values[0]))), values)
    connection_context.connection.commit()
finally:
    cursor.close()
control_df = connection_context.table("PAL_CONTROL_TBL")

print(control_df.head(5).collect())

   LEFT  RIGHT
0   3.0    NaN
1   6.0    6.0
2   6.0    6.0
3   6.0    6.0
4   6.0    6.0


In [11]:
result = distribution_fit(control_df, distr_type = "weibull", optimal_method = "maximum_likelihood", censored=True)
print(result[0].collect())

               NAME    VALUE
0  DISTRIBUTIONNAME  WEIBULL
1             SCALE  31.2021
2             SHAPE  1.43217


In [12]:
print(result[1].collect())

       STAT_NAME  STAT_VALUE
0  LOGLIKELIHOOD  -54.022223


**Step 3**

Get the CDF (cumulative distribution function) of Weibull distribution for the linoleic acid treatment data:

In [13]:
cursor = connection_context.connection.cursor()
try:
    cursor.execute("DROP TABLE PAL_DISTRPROB_DATA_TBL")
except:
    pass
cursor.execute('CREATE COLUMN TABLE PAL_DISTRPROB_DATA_TBL (\"DATACOL\" DOUBLE);')
values = [(6,),(8,),(12,),(20,),(24,),(30,),(42,)]
try:
    cursor.executemany("INSERT INTO " +
                       "{} VALUES ({})".format('PAL_DISTRPROB_DATA_TBL',
                       ', '.join(['?']*len(values[0]))), values)
    connection_context.connection.commit()
finally:
    cursor.close()
distri_prob_df = connection_context.table("PAL_DISTRPROB_DATA_TBL")

print(distri_prob_df.collect())

   DATACOL
0      6.0
1      8.0
2     12.0
3     20.0
4     24.0
5     30.0
6     42.0


Invoke CDF and show the result:

In [14]:
from hana_ml.algorithms.pal.stats import cdf
distr_info = {'name' : 'weibull', 'shape' : 1.40528, 'scale': 36.3069}
result = cdf(distri_prob_df, distr_info, complementary=False)
print(result.collect())

   DATACOL  PROBABILITY
0      6.0     0.076580
1      8.0     0.112515
2     12.0     0.190245
3     20.0     0.351182
4     24.0     0.428183
5     30.0     0.534573
6     42.0     0.706874


**Step 4**

Get the CDF (cumulative distribution function) of Weibull distribution for the control treatment data:

In [15]:
cursor = connection_context.connection.cursor()
try:
    cursor.execute("DROP TABLE PAL_DISTRPROB_DATA_TBL")
except:
    pass
cursor.execute('CREATE COLUMN TABLE PAL_DISTRPROB_DATA_TBL (\"DATACOL\" DOUBLE);')
values = [(6,),(10,),(12,),(24,),(32,)]
try:
    cursor.executemany("INSERT INTO " +
                       "{} VALUES ({})".format('PAL_DISTRPROB_DATA_TBL',
                       ', '.join(['?']*len(values[0]))), values)
    connection_context.connection.commit()
finally:
    cursor.close()
distri_prob_df = connection_context.table("PAL_DISTRPROB_DATA_TBL")

print(distri_prob_df.collect())

   DATACOL
0      6.0
1     10.0
2     12.0
3     24.0
4     32.0


Invoke CDF and show the result:

In [16]:
distr_info = {'name' : 'weibull', 'shape' :  1.71902, 'scale': 20.444}
result = cdf(distri_prob_df, distr_info, complementary=False)
print(result.collect())

   DATACOL  PROBABILITY
0      6.0     0.114457
1     10.0     0.253607
2     12.0     0.329794
3     24.0     0.732173
4     32.0     0.884698


## Drop Tables and Close HANA Connection

In [17]:
cursor = connection_context.connection.cursor()
try:
    cursor.execute("DROP TABLE PAL_TRIAL_DATA_TBL")
    cursor.execute("DROP TABLE PAL_DISTRPROB_DATA_TBL")
    cursor.execute("DROP TABLE PAL_LINO_ACID_TBL")
    cursor.execute("DROP TABLE PAL_CONTROL_TBL")
except:
    pass
connection_context.close()